In [134]:
import os
from glob import glob
from datetime import datetime

import pyart
from matplotlib import pyplot as plt
import cartopy.crs as ccrs # A toolkit for map projections
import numpy as np
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.dates as mdates

import tqdm
import dask
import dask.bag as db
from dask.diagnostics import ProgressBar

import rainrate

In [135]:
img_root = '/g/data/kl02/jss548/PST/polarimetric_rain/img'
vol_root = '/g/data/kl02/jss548/PST/polarimetric_rain/cfradial'
grid_root = '/g/data/kl02/jss548/PST/polarimetric_rain/rf3_instant_rainrate'
date_str = '20200119'
radar_name = '02'
plot_range_km = 100
NCPU = 8
band = 'C'
sweep = 2

In [143]:
def chunks(l, n):
    """
    Yield successive n-sized chunks from l.
    From http://stackoverflow.com/a/312464
    """
    for i in range(0, len(l), n):
        yield l[i:i + n]

def plot_ppi(radar, display, subplot_id, fig, field_name, sweep, vmin, vmax):
    #get lat lon
    lon_range = 1.0
    lat_range = 1.0
    radar_lon = radar.longitude['data']
    radar_lat = radar.latitude['data']
    lon_min = radar_lon-lon_range
    lon_max = radar_lon+lon_range
    lat_min = radar_lat-lat_range
    lat_max = radar_lat+lat_range    
    
    #create subplot
    ax = fig.add_subplot(subplot_id, projection=ccrs.PlateCarree())
    ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
    #plot data
    display.plot_ppi_map(field_name, sweep, 
                       colorbar_flag=True,
                       vmin=vmin, vmax=vmax, cmap='pyart_HomeyerRainbow',
                       resolution='10m')
    #add range rings
    display.plot_range_rings(np.linspace(0,150,6, dtype=np.float), ax=ax, col='k', ls='-', lw=0.5)
    
    #Now we add lat lon lines
    gl = display.ax.gridlines(draw_labels=True,
                              linewidth=1, color='gray', alpha=0.5,
                              linestyle='--')
    gl.xlabel_style = {'size': 10}
    gl.ylabel_style = {'size': 10}
    gl.xlabels_top = False
    gl.ylabels_right = False
    
def ppi_manager(vol_ffn):
    #print('processing',vol_ffn)
    
    #read volume
    radar = pyart.io.read_cfradial(vol_ffn)
    display = pyart.graph.RadarMapDisplay(radar)
    
    #create figure
    fig = plt.figure(figsize=[15,15], facecolor='w')
    #plot data
    plot_ppi(radar, display, 221, fig, 'PHIDP_VAL', 0, 0, 90)
    plot_ppi(radar, display, 222, fig, 'corrected_reflectivity', 0, 10, 60)
    plot_ppi(radar, display, 223, fig, 'specific_attenuation', 0, 0, 0.04)
    plot_ppi(radar, display, 224, fig, 'hybrid_rainrate', 0, 0, 70)

    #save image to file
    #create paths
    img_path = f'{img_root}/ppi/{radar_name}/{date_str}'
    if not os.path.exists(img_path):
        try:
            os.makedirs(img_path)
        except:
            pass
    img_fn = f'{os.path.basename(vol_ffn)[:-3]}_{plot_range_km}km.png'  #this filename should match
    img_ffn = f'{img_path}/{img_fn}'
    
    plt.savefig(img_ffn, dpi=100)  # Saving figure. 
    plt.close()  # Release memory 
    fig.clf()  # Clear figure 
    
def plot_grid(grid_ffn):
    #read grid
    nc = NetCDFFile(grid_ffn)
    rainrate = nc.variables['rain_rate'][:] # rainrate
    x = nc.variables['x'][:] # x
    y = nc.variables['y'][:] # y
    x_grid,y_grid=np.meshgrid(x,y)
    #create figure
    fig = plt.figure(figsize=[15,12], facecolor='w')    
    #plot
    im = plt.pcolormesh(x_grid, y_grid, rainrate, cmap='hot_r', vmin=0, vmax=100)
    cbar = fig.colorbar(im)
    cbar.set_label('Rainrate (mm/hr)', rotation=270)
    plt.grid()
    plt.xlabel('Distance E-W (km)')
    plt.ylabel('Distance N-S (km)')
    #close file
    nc.close()
    
    #save grid
    #create paths
    img_path = f'{img_root}/rf3_grid/{radar_name}/{date_str}'
    if not os.path.exists(img_path):
        try:
            os.makedirs(img_path)
        except:
            pass
    img_fn = f'{os.path.basename(grid_ffn)[:-3]}.png'  #this filename should match
    img_ffn = f'{img_path}/{img_fn}'
    
    plt.savefig(img_ffn, dpi=100)  # Saving figure. 
    plt.close()  # Release memory 
    fig.clf()  # Clear figure 
    
def load_alpha(vol_ffn_list):
    
    #load alpha list
    alpha_list = []
    alpha_method_list = []
    dt_list = []
    for vol_ffn in tqdm. tqdm(vol_ffn_list):
        #read data
        nc = NetCDFFile(vol_ffn)
        dt_list.append(datetime.strptime(nc.variables['time'].units[14:], '%Y-%m-%dT%H:%M:%SZ'))
        alpha_list.append(nc.variables['specific_attenuation'].alpha)
        alpha_method_list.append(nc.variables['specific_attenuation'].alpha_method)
        nc.close()
        
    return dt_list, alpha_list, alpha_method_list

def plot_alpha(dt_list, alpha_list, alpha_method_list, date_str):
    
    myFmt = mdates.DateFormatter('%H')
    
    #setup figure
    fig = plt.figure(figsize=[10,5])
    ax1 = plt.subplot(111)
    ax2 = ax1.twinx()
    #plot alpha
    ax1.plot(dt_list, alpha_list, 'b-')
    ax1.set_ylim([0,0.05])
    ax1.set_ylabel(r"$ \alpha (dB deg^{-1})$", color='b')
    #plot method on secondary axis
    ax2.plot(dt_list, alpha_method_list, 'r-')
    ax2.set_ylim([0,5.1])
    ax2.set_ylabel('alpha technique', color='r')
    #annotations
    ax1.set_xlabel('Hour (UTC)')
    plt.title('Alpha timeseries for ' + dt_list[0].strftime('%Y-%m-%d'))
    ax1.grid()
    ax1.xaxis.set_major_formatter(myFmt)
    
    #save plot
    #create paths
    img_path = f'{img_root}/alpha_ts/{radar_name}/'
    if not os.path.exists(img_path):
        try:
            os.makedirs(img_path)
        except:
            pass
    img_fn = f'{radar_name}_{date_str}_alpha_ts.png'  #this filename should match
    img_ffn = f'{img_path}/{img_fn}'
    
    plt.savefig(img_ffn, dpi=100)  # Saving figure. 
    plt.close()  # Release memory 
    fig.clf()  # Clear figure 

In [140]:
#plot volumes
vol_ffn_list = sorted(glob(f'{vol_root}/{radar_name}/{date_str}/*.nc'))

# for vol_ffn in vol_ffn_list[60:]:
#     ppi_manager(vol_ffn)
#     break

#run matching
i            = 0
n_files      = len(vol_ffn_list)   
for flist_chunk in chunks(vol_ffn_list, NCPU):
    bag = db.from_sequence(flist_chunk).map(ppi_manager)
    _ = bag.compute()
    i += NCPU
    del bag
    print('processed: ' + str(round(i/n_files*100,2)))


processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_000028.vol.nc
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_004231.vol.nc
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_000628.vol.nc
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_003031.vol.nc
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_001228.vol.nc
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_001828.vol.nc
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_002428.vol.nc
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_003628.vol.nc
processed: 3.33
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_20200119_004828.vol.nc
processing /g/data/kl02/jss548/PST/polarimetric_rain/cfradial/02/20200119/02_2020011

In [141]:
#plot grid
grid_ffn_list = sorted(glob(f'{grid_root}/{radar_name}/{date_str}/*.nc'))

# for grid_ffn in grid_ffn_list[60:]:
#     plot_grid(grid_ffn)
#     break
    
#run matching
i            = 0
n_files      = len(grid_ffn_list)   
for flist_chunk in chunks(grid_ffn_list, NCPU):
    bag = db.from_sequence(flist_chunk).map(plot_grid)
    _ = bag.compute()
    i += NCPU
    del bag
    print('processed: ' + str(round(i/n_files*100,2)))    

processed: 3.33
processed: 6.67
processed: 10.0
processed: 13.33
processed: 16.67
processed: 20.0
processed: 23.33
processed: 26.67
processed: 30.0
processed: 33.33
processed: 36.67
processed: 40.0
processed: 43.33
processed: 46.67
processed: 50.0
processed: 53.33
processed: 56.67
processed: 60.0
processed: 63.33
processed: 66.67
processed: 70.0
processed: 73.33
processed: 76.67
processed: 80.0
processed: 83.33
processed: 86.67
processed: 90.0
processed: 93.33
processed: 96.67
processed: 100.0


In [144]:
#plot alpha timeseries
vol_ffn_list = sorted(glob(f'{vol_root}/{radar_name}/{date_str}/*.nc'))
dt_list, alpha_list, alpha_method_list = load_alpha(vol_ffn_list)
plot_alpha(dt_list, alpha_list, alpha_method_list, date_str)


100%|██████████| 240/240 [00:15<00:00, 15.94it/s]
